## PREPARING DATA FOR RAG

# installation de tous les packages requis



In [ ]:
%%capture
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install "torch>=2.1.1" -U

In [ ]:
!pip install -q python-dotenv==1.0.0

!pip install -q pandas==1.5.3
!pip install -q pypdf==3.12.0
!pip install -q jq==1.4.1
!pip install -q nltk==3.8.1
!pip install -q tiktoken==0.4.0
!pip install -q  sentencepiece==0.1.99
!pip install -q sentence-transformers==2.2.2
!pip install -q cohere==4.11.2
!pip install -q faiss-cpu==1.7.4
!pip install -q chromadb==0.3.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 810.5 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.4 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
argilla 1.29.0 requires typer<0.10.0,>=0.6.0, but you have typer 0.12.3 which is incompatible.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [ ]:
!pip install transformers

In [ ]:
!pip install langchain
!pip install langchain-community

# Import all required libraries

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
import re
import pandas as pd
import os

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


 load the pdf that contains our data

In [ ]:
from langchain.document_loaders import PyPDFLoader
llm_loader = PyPDFLoader("/content/A propos de Djezzy (27).pdf")
pages = llm_loader.load_and_split()

CHUNKING

divide our pdf into pieces of document each piece contains a paragraph that talks about an offer or a service (in our pdf we put a separator between the paragraphs)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
import nltk
nltk.download('punkt')

text_splitter = CharacterTextSplitter(
    separator="separatoor",
    is_separator_regex=False,
    # Use the separator of your choice
    chunk_size=200,
    chunk_overlap=0
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In our RAG system, each document fragment contains the text of the paragraph as well as metadata, which serves as a key or title for that document. We have assigned metadata to each paragraph so that when keywords are captured in the user's question, we can directly retrieve the paragraphs corresponding to those keywords, which saves a lot of time.

In [ ]:
#load the document that contains the keywords
with open('/content/mots_clés.txt', 'r') as fichier:
    contenu = fichier.read()


mots = contenu.split(',')

for mot in mots:
    print(mot.strip())


tableau_de_mots = [mot.strip() for mot in mots]


print(tableau_de_mots)

A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
scoop
E facture
verso+
parental control
english learning
sms 3lik
callback relay
B2mail
tranquilo
double call
bank transfer
roaming
golden
personal account
*777#
scoop ramadan
ranati ramadan
nokta
VRxperience
anghami
youscribe
premium gaming
clip+
red list
conference call
international service
E flexy
recharge djezzy
flexy
flexily
legend
zid
3ayla
comfort partage
comfort
Maxi
special
bezzaf
izzy
tunisia
hadj
internet access
BIP SMS
phishing
facebook flex
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos de djezzy
A propos 

In [ ]:
len(tableau_de_mots)

108

In [ ]:
documents=["".join([pages[i].page_content for i in range(len(pages))])]
metadatas = [ {"document":mot.strip() }for mot in mots  ]

In [ ]:
#split our pdf from our data using the text splitter
tokens_chunks = text_splitter.create_documents(
    documents, metadatas=metadatas
)

load and use the embedding MODEL

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


put the paragraphs divided into a table where each cell of the table contains a paragraph.
then using the embedding model we store the paragraph embeddings in a table.

In [ ]:
docs_text = [ chunk.page_content for chunk in tokens_chunks ]
docs_embeddings = embedding_llm.embed_documents(docs_text)

In [ ]:
i=0
for d in tokens_chunks:

 d.metadata=metadatas[i]
 i=i+1
 print(d)


page_content='Regarding Djezzy: Djezzy, an Algerian telecommunications \noperator, was established in July 2001. A leader in the field of \nmobile telephony, with over 14 million subscribers as of August \n2022, the company provides a wide  range of services such as \nprepaid, postpaid, internet , as well as value -added services and \nSUT. Since July 2022, when Veon signed the deed of assignment \nof all its shares in the company to the benefit of the National \nInvestment Fund, Djezzy has become a national company. It is \nnow owned by the National Inves tment Fund (FNI) with \n96.57% yo and by Cevital with 3.43%. Therefore, Djezzy is \ncurrently fully controlled by two Algerian shareholders: the \nNational Investment Fund (FNI) with 96.57% and Cevital with \n3.43%. Djezzy GSM has marked several significant milestones \nthroughout its journey. It all began on July 30, 2001, when the \noperator obtained its 2G license, thus laying the groundwork for \nits mobile telephony services in 

Vector Stores | FAISS

In [ ]:
from langchain.vectorstores import FAISS
vector_db = FAISS.from_documents(tokens_chunks, embedding_llm)

save the table that contains the paragraphs of our data and also the table that contains the embedding of these paragraphs

In [ ]:
import json


docs_text = [ chunk.page_content for chunk in tokens_chunks ]
docs_embeddings = embedding_llm.embed_documents(docs_text)


with open('docs_text.json', 'w', encoding='utf-8') as f:
    json.dump(docs_text, f, ensure_ascii=False, indent=4)


with open('docs_embeddings.json', 'w', encoding='utf-8') as f:
    json.dump(docs_embeddings, f, ensure_ascii=False, indent=4)

load vector representation

In [ ]:


vector_db.save_local("d")

# Check if the file has been saved
os.path.exists("d")

True